In [1]:
# SQL (SQLAlchemy)

In [2]:
%pip install -qU langchain-community SQLAlchemy langchain-openai

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
################ 사용 방법 ################ 
'''
1. Session Id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자입니다.
2. Connection string - 데이터베이스 연결을 지정하는 문자열입니다. 
   이 문자열은 SQLAlchemy의 create_engine 함수에 전달됩니다.

- SQLChatMessageHistory 클래스를 사용하여 대화 메시지 기록 관리
- session_id와 connection_string을 인자로 전달하여 SQLChatMessageHistory 인스턴스 생성
- add_user_message 메서드를 사용하여 사용자 메시지를 대화 기록에 추가
- add_ai_message 메서드를 사용하여 AI 메시지를 대화 기록에 추가
'''

'\n1. Session Id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자입니다.\n2. Connection string - 데이터베이스 연결을 지정하는 문자열입니다. \n   이 문자열은 SQLAlchemy의 create_engine 함수에 전달됩니다.\n\n- SQLChatMessageHistory 클래스를 사용하여 대화 메시지 기록 관리\n- session_id와 connection_string을 인자로 전달하여 SQLChatMessageHistory 인스턴스 생성\n- add_user_message 메서드를 사용하여 사용자 메시지를 대화 기록에 추가\n- add_ai_message 메서드를 사용하여 AI 메시지를 대화 기록에 추가\n'

In [11]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달합니다.
chat_message_history = SQLChatMessageHistory(
    session_id="sql_chat_history", connection_string="sqlite:///sqlite.db"
)

# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "Hi! My name is Teddy. I am a AI programmer. Nice to meet you!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("Hi Teddy! Nice to meet you too!")

C:\ProgramData\Anaconda3\envs\langchainenv\Lib\site-packages\langchain_community\chat_message_histories\sql.py:186: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. Use connection instead.
  warn_deprecated(


In [12]:
# 채팅 메시지 기록의 메시지들
chat_message_history.messages

[HumanMessage(content='Hi! My name is Teddy. I am a AI programmer. Nice to meet you!'),
 AIMessage(content='Hi Teddy! Nice to meet you too!')]

In [13]:
### Chain 에 적용 ###

In [14]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [15]:
'''
ChatPromptTemplate을 사용하여 프롬프트 템플릿 정의
    - "system" 메시지: "You are a helpful assistant."
    - MessagesPlaceholder를 사용하여 "history" 변수를 프롬프트에 포함
    - "human" 메시지: "{question}"
프롬프트 템플릿과 ChatOpenAI 모델을 파이프라인으로 연결하여 chain 생성
'''

'\nChatPromptTemplate을 사용하여 프롬프트 템플릿 정의\n    - "system" 메시지: "You are a helpful assistant."\n    - MessagesPlaceholder를 사용하여 "history" 변수를 프롬프트에 포함\n    - "human" 메시지: "{question}"\n프롬프트 템플릿과 ChatOpenAI 모델을 파이프라인으로 연결하여 chain 생성\n'

In [24]:
import openai

prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지를 설정하여 어시스턴트의 역할을 정의합니다.
        ("system", "You are a helpful assistant."),
        # 이전 대화 내용을 포함하기 위한 플레이스홀더를 추가합니다.
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),  # 사용자의 질문을 입력받는 메시지를 설정합니다.
    ]
)

chain = (
    prompt | ChatOpenAI(api_key = "sk-")
)  # 프롬프트와 ChatOpenAI 모델을 연결하여 체인을 생성합니다.

In [25]:
'''
- RunnableWithMessageHistory 클래스를 사용하여 chain과 메시지 기록 연결
- lambda 함수를 사용하여 session_id에 따라 SQLChatMessageHistory 인스턴스 생성
    - SQLChatMessageHistory는 SQLite 데이터베이스(sqlite.db)에 연결되어 채팅 메시지 기록 저장
- input_messages_key를 "question"으로 설정하여 입력 메시지의 키 지정
- history_messages_key를 "history"로 설정하여 기록 메시지의 키 지정
'''

'\n- RunnableWithMessageHistory 클래스를 사용하여 chain과 메시지 기록 연결\n- lambda 함수를 사용하여 session_id에 따라 SQLChatMessageHistory 인스턴스 생성\n    - SQLChatMessageHistory는 SQLite 데이터베이스(sqlite.db)에 연결되어 채팅 메시지 기록 저장\n- input_messages_key를 "question"으로 설정하여 입력 메시지의 키 지정\n- history_messages_key를 "history"로 설정하여 기록 메시지의 키 지정\n'

In [26]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),  # session_id를 기반으로 SQLChatMessageHistory 객체를 생성하는 람다 함수
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="history",  # 대화 기록 메시지의 키를 "history"로 설정
)

In [27]:
#config 딕셔너리를 정의하고, "configurable" 키 아래에 "session_id" 키-값 쌍 설정

In [28]:
# 세션 ID를 구성하는 곳입니다.
config = {"configurable": {"session_id": "sql_chat_history"}}

In [29]:
# ex) 질문에 이름을 물어보는 질문

In [30]:
# chain_with_history 객체의 invoke 메서드를 호출하여 질문에 대한 답변 생성
# invoke 메서드에는 질문 딕셔너리와 config 설정 전달

In [31]:
# 질문 "Whats my name"과 설정을 사용하여 대화 기록이 있는 체인을 호출합니다.
chain_with_history.invoke({"question": "Whats my name?"}, config=config)

AIMessage(content='Your name is Teddy.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 56, 'total_tokens': 61}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4f24a702-b9cb-42bb-a6c0-db386575c0a4-0', usage_metadata={'input_tokens': 56, 'output_tokens': 5, 'total_tokens': 61})

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
